In [1]:
import os
import sys
import glob
from modeller import *

### From the BLAST results and the MSA of the best homologs, choose several structures to use as templates for the modelling:

##### We keep 2ZTN because it has the best sequence identity among those with e-value=0 that cover the entire protein
##### We add 8PMX because it has the best sequence identity for the P domain of interest.

In [2]:
target = "capsid"
pdbs = ["", ""]
chain = ["", ""]

#### Get the pdb structures

In [3]:
for pdb in pdbs:
    if not glob.glob("../../Structures/{}.pdb".format(pdb)):
        print("downloading {}.pdb".format(pdb))
        os.system("curl -s -f https://files.rcsb.org/download/{}.pdb -o ../../Structures/{}.pdb".format(pdb, pdb))
    else:
        print("{}.pdb already downloaded.".format(pdb))

#### Align the template sequence with the target sequence on MAFFT or CLUSTALW

In [4]:
# the target fasta sequence
with open("../../Sequences/{}.fasta".format(target), "r") as f:
    print(f.read())
for i, pdb in enumerate(pdbs):
    # the template pdb fasta sequences
    with open("../../Sequences/{}_{}.fasta".format(pdb, chain[i]), "r") as f:
        print(f.read())

##### Download the fasta format output
##### Deposit in this directory and name it "alignment.fasta"

#### Convert the CLUSTAL format alignment into pir format:
##### Adjust the pir file if necessary

In [5]:
with open("alignment.fasta", "r") as f:
    data = [el.rstrip("\n") for el in f.readlines()]

alignment = dict()
key = None
value = ""
for el in data:
    if el[0] == ">":
        value = ""
        key = el.lstrip(">")
    else:
        if key is not None:
            if key not in alignment.keys():
                alignment[key] = [el]
            else:
                alignment[key].append(el)
                #value = value + el
            #alignment[key] = value

with open("alignment.pir", "w") as f:
    for k, val in alignment.items():
        f.write(">P1;{}\n".format(k))
        # if target sequence:
        if k == target:
            f.write("sequence:{}:FIRST:A:LAST:::::\n".format(k))
        else:
            f.write("structureX:{}:FIRST:A:LAST:::::\n".format(k))
        for i, el in enumerate(val):
            if i == len(val)-1:
                f.write("{}*\n".format(el))
            else:
                f.write("{}\n".format(el))

#### Build the model using automodel by executing the cells below or by running build_model.py in the directory (recommended).

In [6]:
from modeller.automodel import *

In [10]:
env4 = Environ()
env4.io.atom_files_directory = ['.', '../../Structures/']

#sys._jupyter_stdout = sys.stdout
#sys.stdout = open(os.devnull, 'w') # block prints to avoid overflow of lines in our jupyter notebook.
###

a = AutoModel(env4, 
              alnfile='alignment.pir',
              knowns=tuple(pdbs), 
              sequence=target,
              assess_methods=(assess.DOPE,
                              assess.normalized_dope,
                              assess.GA341))
a.starting_model = 1
a.ending_model = 5
a.make()

###
#sys.stdout = sys._jupyter_stdout #sys.__stdout__ # reenable prints
#print("done.")

### Preparation for analysis

In [13]:
from modeller.scripts import complete_pdb

env5 = Environ()
env5.io.atom_files_directory = ['.', '../../Structures/']
env5.libs.topology.read(file='$(LIB)/top_heav.lib') # read topology
env5.libs.parameters.read(file='$(LIB)/par.lib') # read parameters

### Make DOPE score files for analysis

In [15]:
# read model file
mdl1 = complete_pdb(env5, './capsid.B99990001.pdb')
mdl2 = complete_pdb(env5, './capsid.B99990002.pdb')
mdl3 = complete_pdb(env5, './capsid.B99990003.pdb')
mdl4 = complete_pdb(env5, './capsid.B99990004.pdb')
mdl5 = complete_pdb(env5, './capsid.B99990005.pdb')

sys._jupyter_stdout = sys.stdout
sys.stdout = open(os.devnull, 'w') # block prints to avoid overflow of lines in our jupyter notebook.
###

# Assess with DOPE:
s1 = Selection(mdl1)   # all atom selection
s1.assess_dope(output='ENERGY_PROFILE NO_REPORT', file='capsid_model1.profile',
              normalize_profile=True, smoothing_window=15) # DOPE energy calculation

s2 = Selection(mdl2)   # all atom selection
s2.assess_dope(output='ENERGY_PROFILE NO_REPORT', file='capsid_model2.profile',
              normalize_profile=True, smoothing_window=15) # DOPE energy calculation

s3 = Selection(mdl3)   # all atom selection
s3.assess_dope(output='ENERGY_PROFILE NO_REPORT', file='capsid_model3.profile',
              normalize_profile=True, smoothing_window=15) # DOPE energy calculation

s4 = Selection(mdl4)   # all atom selection
s4.assess_dope(output='ENERGY_PROFILE NO_REPORT', file='capsid_model4.profile',
              normalize_profile=True, smoothing_window=15) # DOPE energy calculation

s5 = Selection(mdl5)   # all atom selection
s5.assess_dope(output='ENERGY_PROFILE NO_REPORT', file='capsid_model5.profile',
              normalize_profile=True, smoothing_window=15) # DOPE energy calculation

###
sys.stdout = sys._jupyter_stdout #sys.__stdout__ # reenable prints
print("done.")